In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys 
import statsmodels as sm
import statsmodels.api as smapi
import statsmodels.regression.linear_model as lm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
sys.path.append(os.path.abspath("/Users/alvaradojo/Documents/Github/iROIEstimator/scripts"))
import Utilities as utils
import Constants as c
%matplotlib inline

In [2]:
project_name = "angular"
directoryPath = "../../exports"
bugs = "{directoryPath}/{project_name}/{project_name}_dataset_BUG.csv".format(directoryPath=directoryPath, project_name=project_name)

cc_columns = [c.VERSION, c.DATE, c.NT_CC, c.NO_CC, c.MODULE_CC, c.LINE_CC, c.T_MODULE, c.T_LINE, c.T_CC]
df = pd.read_csv(bugs, usecols = cc_columns)
df[c.DATE] = pd.to_datetime(df[c.DATE])
df = df.dropna(subset=[c.T_MODULE])
# df.head()

In [3]:
# df.describe()
# df.dtypes

In [4]:
# display correlation matrix
display(df.corr())

,T_Module,T_Line,NT_CC,NO_CC,Module_CC,Line_CC,T_CC
T_Module,1.000000,0.958264,0.330150,0.378968,0.274809,0.338145,-0.074578
T_Line,0.958264,1.000000,0.340862,0.427307,0.232036,0.327403,-0.029472
NT_CC,0.330150,0.340862,1.000000,0.914693,0.797218,0.884715,0.537878
NO_CC,0.378968,0.427307,0.914693,1.000000,0.688748,0.833090,0.474900
Module_CC,0.274809,0.232036,0.797218,0.688748,1.000000,0.929930,0.386832
Line_CC,0.338145,0.327403,0.884715,0.833090,0.929930,1.000000,0.400709
T_CC,-0.074578,-0.029472,0.537878,0.474900,0.386832,0.400709,1.000000


In [5]:
# sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap='RdBu')

In [6]:
# df.plot.scatter(x=c.NT_CC, y=c.LINE_CC, c='red')
# df.plot.scatter(x=c.NO_CC, y=c.LINE_CC, c='green')

In [7]:
# Let's create multiple regression
X = df[[c.NT_CC, c.NO_CC]]
Y = df[c.LINE_CC]
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, test_size=0.25, random_state=0)

model = lm.OLS(y_train, X_train).fit()
predictions = model.predict(X_test)
 
print_model = model.summary()
print(print_model)
# Interpreting Restuls:  https://www.youtube.com/watch?v=U7D1h5bbpcs  

OLS Regression Results                                
Dep. Variable:                Line_CC   R-squared (uncentered):                   0.818
Model:                            OLS   Adj. R-squared (uncentered):              0.816
Method:                 Least Squares   F-statistic:                              543.2
Date:                Sun, 26 Apr 2020   Prob (F-statistic):                    3.32e-90
Time:                        19:28:22   Log-Likelihood:                         -2574.5
No. Observations:                 244   AIC:                                      5153.
Df Residuals:                     242   BIC:                                      5160.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [8]:
data = {c.OBSERVED:y_test, c.PREDICTED:predictions.round(2), c.DIFFERENCE:abs(y_test - predictions).round(2), c.PERCENT_ERROR:(abs(y_test - predictions)/y_test).round(2)}
results = pd.DataFrame(data) 
results[c.VERSION] = np.arange(results.__len__())
# results

In [35]:
print("{0} - {1} - {2} model performance: ".format(project_name, "BUG", c.MODULE_CC))
print(utils.format_perf_metric('Model - R Squared', model.rsquared))
print(utils.format_perf_metric('Model - R Squared Adj', model.rsquared_adj))
# print(utils.format_perf_metric('Model - Mean Squared Error', model.mse_model))
# print(utils.format_perf_metric('Model - Mean Squared Error Residuals', model.mse_resid))
# print(utils.format_perf_metric('Model - Mean Squared Error Total', model.mse_total))
print(utils.format_perf_metric('Pred - Mean Absolute Error', metrics.mean_absolute_error(y_test, predictions)))
print(utils.format_perf_metric('Pred - Mean Squared Error', metrics.mean_squared_error(y_test, predictions)))
print(utils.format_perf_metric('Pred - Root Mean Squared Error', np.sqrt(metrics.mean_squared_error(y_test, predictions))))
pred25 = utils.calculate_PRED(0.25, results)
print(utils.format_PRED("25", pred25))
pred50 = utils.calculate_PRED(0.50, results)
print(utils.format_PRED("50", pred50))

# print(model.tvalues)
# print(model.pvalues)
# print(model.fvalue)
# print(model.f_pvalue)


angular - BUG - Module_CC model performance: 
Model - R Squared: 0.68
Model - R Squared Adj: 0.68
Pred - Mean Absolute Error: 285.81
Pred - Mean Squared Error: 281777.42
Pred - Root Mean Squared Error: 530.83
PRED (25): 20.73%
PRED (50): 32.93%


In [10]:
# ax = results.plot(kind='scatter', x=c.VERSION, y=c.OBSERVED, c='green', label=c.OBSERVED)
# results.plot(kind='scatter', x=c.VERSION, y=c.PREDICTED, c='blue', label=c.PREDICTED, ax=ax)

In [11]:
# partialRegPlot = plt.figure(figsize=(15,8))
# partialRegPlot = sm.graphics.regressionplots.plot_partregress_grid(model, fig=partialRegPlot)

In [12]:
# regPlotNC = plt.figure(figsize=(15,8))
# regPlotNC = sm.graphics.regressionplots.plot_regress_exog(model, c.NT_CC, fig=regPlotNC)

In [13]:
# regPlotNO = plt.figure(figsize=(15,8))
# regPlotNO = sm.graphics.regressionplots.plot_regress_exog(model, c.NO_CC, fig=regPlotNO)

In [14]:
# df.plot.scatter(x=c.NT_CC, y=c.MODULE_CC, c='red')
# df.plot.scatter(x=c.NO_CC, y=c.MODULE_CC, c='green')

In [15]:
# Let's create multiple regression
X = df[[c.NT_CC, c.NO_CC, c.T_MODULE]]
Y = df[c.MODULE_CC]
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, test_size=0.25, random_state=0)
model = lm.OLS(y_train, X_train).fit()
predictions = model.predict(X_test)
 
print_model = model.summary()
print(print_model)

OLS Regression Results                                
Dep. Variable:              Module_CC   R-squared (uncentered):                   0.682
Model:                            OLS   Adj. R-squared (uncentered):              0.678
Method:                 Least Squares   F-statistic:                              172.5
Date:                Sun, 26 Apr 2020   Prob (F-statistic):                    1.03e-59
Time:                        19:28:23   Log-Likelihood:                         -1871.0
No. Observations:                 244   AIC:                                      3748.
Df Residuals:                     241   BIC:                                      3758.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [16]:
data = {c.OBSERVED:y_test, c.PREDICTED:predictions.round(2), c.DIFFERENCE:abs(y_test - predictions).round(2), c.PERCENT_ERROR:(abs(y_test - predictions)/y_test).round(2)}
results = pd.DataFrame(data) 
results[c.VERSION] = np.arange(results.__len__())
# results

In [25]:
print("{0} - {1} - {2} model performance: ".format(project_name, "BUG", c.LINE_CC))

print(utils.format_perf_metric('Model - R Squared', model.rsquared))
print(utils.format_perf_metric('Model - R Squared Adj', model.rsquared_adj))
print(utils.format_perf_metric('Model - Mean Squared Error', model.mse_model))
print(utils.format_perf_metric('Model - Mean Squared Error Residuals', model.mse_resid))
print(utils.format_perf_metric('Model - Mean Squared Error Total', model.mse_total))
print(utils.format_perf_metric('Pred - Mean Absolute Error', metrics.mean_absolute_error(y_test, predictions)))
print(utils.format_perf_metric('Pred - Mean Squared Error', metrics.mean_squared_error(y_test, predictions)))
print(utils.format_perf_metric('Pred - Root Mean Squared Error', np.sqrt(metrics.mean_squared_error(y_test, predictions))))
print(utils.format_perf_metric('Pred - Mean Squared Error', metrics.mean_squared_error(y_test, predictions)))
print(utils.format_perf_metric('Pred - Root Mean Squared Error', np.sqrt(metrics.mean_squared_error(y_test, predictions))))
pred25 = utils.calculate_PRED(0.25, results)
print(utils.format_PRED("25", pred25))
pred50 = utils.calculate_PRED(0.50, results)
print(utils.format_PRED("50", pred50))

angular - BUG - Line_CC model performance: 
Model - R Squared: 0.68
Model - R Squared Adj: 0.68
Model - Mean Squared Error: 46772508.7
Model - Mean Squared Error Residuals: 271152.71
Model - Mean Squared Error Total: 842890.69
Pred - Mean Absolute Error: 285.81
Pred - Mean Squared Error: 281777.42
Pred - Root Mean Squared Error: 530.83
Pred - Mean Squared Error: 281777.42
Pred - Root Mean Squared Error: 530.83
PRED (25): 20.73%
PRED (50): 32.93%


In [18]:
# ax = results.plot(kind='scatter', x=c.VERSION, y=c.OBSERVED, c='green', label=c.OBSERVED)
# results.plot(kind='scatter', x=c.VERSION, y=c.PREDICTED, c='blue', label=c.PREDICTED, ax=ax)

In [19]:
# partialRegPlot = plt.figure(figsize=(15,8))
# partialRegPlot = sm.graphics.regressionplots.plot_partregress_grid(model, fig=partialRegPlot)

In [20]:
# regPlotNC = plt.figure(figsize=(15,8))
# regPlotNC = sm.graphics.regressionplots.plot_regress_exog(model, c.NT_CC, fig=regPlotNC)

In [21]:
# regPlotNO = plt.figure(figsize=(15,8))
# regPlotNO = sm.graphics.regressionplots.plot_regress_exog(model, c.NO_CC, fig=regPlotNO)